In [1]:
import pandas as pd
import sqlite3

# Olympic Athletes & Events (ETL): Extract

In [2]:
#import countries competing in Olympics
noc_regions = pd.read_csv("../Data/01_noc_regions.csv")
print(noc_regions.shape)
noc_regions.head()

(230, 3)


,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN


In [3]:
#import list of athletes competing in events
athlete_events = pd.read_csv("../Data/01_athlete_events.csv")
athlete_events.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


# Olympic Athletes & Event Results (ETL): Transform

In [4]:
#drop extraneous country columns
noc_regions = noc_regions.drop(['notes'], axis=1)
print(noc_regions.shape)
noc_regions.head()

(230, 2)


,NOC,region
0,AFG,Afghanistan
1,AHO,Curacao
2,ALB,Albania
3,ALG,Algeria
4,AND,Andorra


In [5]:
#drop extraneous athlete and event columns
athlete_events = athlete_events.drop(['Name',
                                      'Team',
                                      'Games',
                                      'Season',
                                      'City',
                                      'Event',
                                      'Medal',
                                      'ID'], axis=1)

print(athlete_events.shape)
athlete_events.head()

(271116, 7)


,Sex,Age,Height,Weight,NOC,Year,Sport
0,M,24.0,180.0,80.0,CHN,1992,Basketball
1,M,23.0,170.0,60.0,CHN,2012,Judo
2,M,24.0,NaN,NaN,DEN,1920,Football
3,M,34.0,NaN,NaN,DEN,1900,Tug-Of-War
4,F,21.0,185.0,82.0,NED,1988,Speed Skating


In [6]:
#create clean athletes table for combination of years and competing countries
athletes = pd.DataFrame()

years = athlete_events['Year'].unique().tolist()
years.sort()

NOCs = athlete_events['NOC'].unique().tolist()
NOCs.sort()

for year in years:
    for NOC in NOCs:
        athletes = athletes.append({'Year': year, 'NOC': NOC}, ignore_index=True)
    
athletes['Year'] = athletes['Year'].astype(int)
print(athletes.shape)
athletes.head()

(8050, 2)


,NOC,Year
0,AFG,1896
1,AHO,1896
2,ALB,1896
3,ALG,1896
4,AND,1896


In [7]:
#count total athletes by year and country and merge into athletes table
counts = athlete_events.groupby(['Year', 'NOC']).count()
counts.rename(columns={'Sex': 'Total_Athletes'}, inplace=True)
counts = counts.drop(['Age', 'Height', 'Weight', 'Sport'], axis=1)

athletes = pd.merge(athletes, counts,  how='left', left_on=['Year','NOC'], right_on = ['Year','NOC'])

#drop row in years in which country did not compete
athletes = athletes.dropna()
print(athletes.shape)
athletes.head()

(3305, 3)


,NOC,Year,Total_Athletes
12,AUS,1896,5.0
13,AUT,1896,8.0
55,DEN,1896,15.0
69,FRA,1896,26.0
74,GBR,1896,25.0


In [9]:
#count total athletes by year
total_annual_athletes = athletes.groupby(['Year']).sum()
total_annual_athletes.rename(columns={'Total_Athletes': 'Total_Annual_Athletes'}, inplace=True)
total_annual_athletes

,Total_Annual_Athletes
Year,
1896,380.0
1900,1936.0
1904,1301.0
1906,1733.0
1908,3101.0
1912,4040.0
1920,4292.0
1924,5693.0
1928,5574.0


In [10]:
#count athlete percentage of total athletes by year and country and merge into athletes table
athlete_percentage = total_annual_athletes
athlete_percentage.rename(columns={'Total_Annual_Athletes': 'Athlete_Percentage'}, inplace=True)

athletes = pd.merge(athletes, athlete_percentage,  how='left', left_on=['Year'], right_on = ['Year'])
athletes['Athlete_Percentage'] = athletes['Total_Athletes'] / athletes['Athlete_Percentage'] * 100
athletes.head()

,NOC,Year,Total_Athletes,Athlete_Percentage
0,AUS,1896,5.0,1.315789
1,AUT,1896,8.0,2.105263
2,DEN,1896,15.0,3.947368
3,FRA,1896,26.0,6.842105
4,GBR,1896,25.0,6.578947


In [11]:
#find females by year and country and merge into athletes table
females = athlete_events[athlete_events.Sex == 'F']
females = females.groupby(['Year', 'NOC']).count()
females = females.drop(['Age', 'Height', 'Weight', 'Sport'], axis=1)
females.rename(columns={'Sex': 'Females'}, inplace=True)

athletes = pd.merge(athletes, females,  how='left', left_on=['Year','NOC'], right_on = ['Year','NOC'])
athletes = athletes.fillna(0)
athletes['Female_Ratio'] = athletes['Females'] / athletes['Total_Athletes']
athletes.head()

,NOC,Year,Total_Athletes,Athlete_Percentage,Females,Female_Ratio
0,AUS,1896,5.0,1.315789,0.0,0.0
1,AUT,1896,8.0,2.105263,0.0,0.0
2,DEN,1896,15.0,3.947368,0.0,0.0
3,FRA,1896,26.0,6.842105,0.0,0.0
4,GBR,1896,25.0,6.578947,0.0,0.0


In [12]:
#find sports competed by year and country and merge into athletes table
sports = athlete_events.groupby(['Year', 'NOC', 'Sport']).nunique()
sports = sports.drop(['Age', 'Height', 'Weight', 'Sex', 'NOC', 'Year'], axis=1)
sports = pd.DataFrame(sports.groupby(['Year', 'NOC']).count())
sports.rename(columns={'Sport': 'Sports_Competed'}, inplace=True)

athletes = pd.merge(athletes, sports,  how='left', left_on=['Year','NOC'], right_on = ['Year','NOC'])
athletes = athletes.fillna(0)
athletes.head()

,NOC,Year,Total_Athletes,Athlete_Percentage,Females,Female_Ratio,Sports_Competed
0,AUS,1896,5.0,1.315789,0.0,0.0,2
1,AUT,1896,8.0,2.105263,0.0,0.0,3
2,DEN,1896,15.0,3.947368,0.0,0.0,5
3,FRA,1896,26.0,6.842105,0.0,0.0,6
4,GBR,1896,25.0,6.578947,0.0,0.0,7


In [13]:
#count total sports by year
total_annual_sports = athlete_events.groupby(['Year', 'Sport'])['Sport'].nunique()
total_annual_sports = pd.DataFrame(total_annual_sports.groupby('Year').count())
total_annual_sports.rename(columns={'Sport': 'Total_Annual_Sports'}, inplace=True)
total_annual_sports

,Total_Annual_Sports
Year,
1896,9
1900,20
1904,18
1906,13
1908,24
1912,17
1920,25
1924,30
1928,25


In [14]:
#count sports percentage of total sports by year and country and merge into athletes table
sports_percentage = total_annual_sports
sports_percentage.rename(columns={'Total_Annual_Sports': 'Sports_Percentage'}, inplace=True)

athletes = pd.merge(athletes, sports_percentage,  how='left', left_on=['Year'], right_on = ['Year'])
athletes['Sports_Percentage'] = athletes['Sports_Competed'] / athletes['Sports_Percentage'] * 100
athletes.head()

,NOC,Year,Total_Athletes,Athlete_Percentage,Females,Female_Ratio,Sports_Competed,Sports_Percentage
0,AUS,1896,5.0,1.315789,0.0,0.0,2,22.222222
1,AUT,1896,8.0,2.105263,0.0,0.0,3,33.333333
2,DEN,1896,15.0,3.947368,0.0,0.0,5,55.555556
3,FRA,1896,26.0,6.842105,0.0,0.0,6,66.666667
4,GBR,1896,25.0,6.578947,0.0,0.0,7,77.777778


In [15]:
#find mean age, height, weight by year and country and merge into athletes table
means = athlete_events.groupby(['Year', 'NOC']).mean()
means

athletes = pd.merge(athletes, means,  how='left', left_on=['Year','NOC'], right_on = ['Year','NOC'])
athletes = athletes.dropna()
athletes.head()

,NOC,Year,Total_Athletes,Athlete_Percentage,Females,Female_Ratio,Sports_Competed,Sports_Percentage,Age,Height,Weight
4,GBR,1896,25.0,6.578947,0.0,0.0,7,77.777778,26.526316,188.000000,102.000000
5,GER,1896,94.0,24.736842,0.0,0.0,6,66.666667,24.261905,163.681818,64.714286
6,GRE,1896,148.0,38.947368,0.0,0.0,9,100.000000,21.235294,175.666667,78.000000
11,USA,1896,27.0,7.105263,0.0,0.0,3,33.333333,23.074074,179.875000,72.461538
13,AUS,1900,6.0,0.309917,0.0,0.0,2,10.000000,22.000000,178.000000,71.000000


In [16]:
#set types in athletes table
athletes['Total_Athletes'] = athletes['Total_Athletes'].astype(int)
athletes['Females'] = athletes['Females'].astype(int)
athletes['Sports_Competed'] = athletes['Sports_Competed'].astype(int)

print(athletes.shape)
athletes.head()

(3091, 11)


,NOC,Year,Total_Athletes,Athlete_Percentage,Females,Female_Ratio,Sports_Competed,Sports_Percentage,Age,Height,Weight
4,GBR,1896,25,6.578947,0,0.0,7,77.777778,26.526316,188.000000,102.000000
5,GER,1896,94,24.736842,0,0.0,6,66.666667,24.261905,163.681818,64.714286
6,GRE,1896,148,38.947368,0,0.0,9,100.000000,21.235294,175.666667,78.000000
11,USA,1896,27,7.105263,0,0.0,3,33.333333,23.074074,179.875000,72.461538
13,AUS,1900,6,0.309917,0,0.0,2,10.000000,22.000000,178.000000,71.000000


# Olympic Athletes & Event Results (ETL): Load

In [17]:
#creating SQL connection
conn = sqlite3.connect('../Data/olympics.db')
c = conn.cursor()

#function to close connection
def close_c_conn():
    c.close()
    conn.close()

In [18]:
#create regions table in SQL
noc_regions.to_sql('regions', conn, if_exists='replace')

#check SQL regions table
regions = pd.read_sql_query("SELECT * FROM regions;", conn)
print(regions.shape)
regions.head()

(230, 3)


,index,NOC,region
0,0,AFG,Afghanistan
1,1,AHO,Curacao
2,2,ALB,Albania
3,3,ALG,Algeria
4,4,AND,Andorra


In [20]:
#create athletes table in SQL
athletes.to_sql('athletes', conn, if_exists='replace')

#check SQL athletes table
athletes = pd.read_sql_query("SELECT * FROM athletes;", conn)
print(athletes.shape)
athletes.head()

(3091, 13)


,level_0,index,NOC,Year,Total_Athletes,Athlete_Percentage,Females,Female_Ratio,Sports_Competed,Sports_Percentage,Age,Height,Weight
0,0,4,GBR,1896,25,6.578947,0,0.0,7,77.777778,26.526316,188.000000,102.000000
1,1,5,GER,1896,94,24.736842,0,0.0,6,66.666667,24.261905,163.681818,64.714286
2,2,6,GRE,1896,148,38.947368,0,0.0,9,100.000000,21.235294,175.666667,78.000000
3,3,11,USA,1896,27,7.105263,0,0.0,3,33.333333,23.074074,179.875000,72.461538
4,4,13,AUS,1900,6,0.309917,0,0.0,2,10.000000,22.000000,178.000000,71.000000


In [21]:
#close connection
close_c_conn()